In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
import joblib

In [ ]:
df = pd.read_csv('../../datasets/CICIOT2023/full_version/lighter_CICIOT2023.csv')

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

/tmp/ipykernel_36234/3510967236.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../datasets/CICIOT2023/full_version/merged.csv')


### Dropping useless columns

In [21]:
print(df)

              flow_duration Header_Length Protocol Type Duration  \
0                       0.0          54.0           6.0     64.0   
1                       0.0           0.0           1.0     64.0   
2                   0.07496       31247.0          17.0     64.0   
3                       0.0           0.0           1.0     64.0   
4        3.5216370820999146      171704.8           8.2    117.9   
...                     ...           ...           ...      ...   
2103094                 0.0           0.0           1.0     64.0   
2103095                 0.0          54.0           6.0     64.0   
2103096            0.242968      34894.19         16.61    66.74   
2103097            0.185182         65.52          6.11    64.64   
2103098                 0.0          54.0           6.0     64.0   

                      Rate              Srate Drate fin_flag_number  \
0                2097152.0          2097152.0   0.0             0.0   
1               172.310827         172.31

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Drop useless columns

useless_column = []
# columns where we have always the same value
for col in df.columns:
    if df[col].nunique() == 1:
        useless_column.append(col)

print(f"Useless columns : {useless_column}")

cols_to_drop = [
    "Protocol Type",
    "Magnitue", "Radius", "Covariance", "Variance", "Weight",
]
df.drop(columns=cols_to_drop)
df.drop(columns=useless_column, inplace=True)

Useless columns : []


### Separing datas and labels

In [23]:
df.columns = df.columns.str.strip() # to clean spaces on the columns names
X = df.drop(columns=['label']) # without labels
Y = df['label'] # just labels

Y = Y.apply(lambda x: 0 if x == 'BenignTraffic' else 1)
print(Y.value_counts())

label
1    1989497
0     113602
Name: count, dtype: int64


### Encoding categorical data
Difference between Label and OneHot : \
                                      - Label gives a number in int for each line (simple but the model could misunderstand the difference between numbers (priorities issues))\
                                      - OneHot encodes in binary columns (no hierarchy between column but a lot of columns if they are a lot of categories to label)

In [24]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# no need of OneHotEncoder because all columns are numerical columns

### Splitting into training set and test set
Split dataset into training and testing sets (70/30)


In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0, stratify=Y)
print(X_test)

              flow_duration Header_Length Protocol Type Duration  \
1343662                 0.0          54.0           6.0     64.0   
1437992            0.050684       45272.5         16.84    65.91   
654961             0.005736       39867.4           6.0     56.0   
203682   0.0527489209175109       35275.0          17.0     64.0   
1794846                 0.0          54.0           6.0     64.0   
...                     ...           ...           ...      ...   
672688                  0.0          54.0           6.0     64.0   
516857   0.0556870889663696         61.56          5.94    63.36   
1455834            4.560022        106.92          5.95     64.0   
1569223                 0.0           0.0           1.0     64.0   
93751     2.275368204116821     621914.86         16.68     64.0   

                       Rate               Srate Drate fin_flag_number  \
1343662             6.62172             6.62172   0.0             0.0   
1437992        17883.265083        17

### Missing datas
useless here because no missing datas

In [ ]:
from sklearn.impute import SimpleImputer # to handle missing data

#here there is no missing data so we don't have to manage this

X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')
#because numeric values are not really numeric in the csv, so we convert them


X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
numeric_cols = X_train.select_dtypes(include=[np.number]).columns


imputer = SimpleImputer(strategy='mean')
X_train[numeric_cols] = imputer.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = imputer.transform(X_test[numeric_cols])

Colonnes numériques détectées : ['flow_duration', 'Header_Length', 'Protocol Type', 'Duration', 'Rate', 'Srate', 'Drate', 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'ack_count', 'syn_count', 'fin_count', 'urg_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC', 'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number', 'Magnitue', 'Radius', 'Covariance', 'Variance', 'Weight']
X_train shape : (1472169, 46)
X_train[numeric_cols] shape : (1472169, 46)


### Feature Scaling
To have the same scale between each columns (for instance, `Flow Duration` is way bigger than `Tot Fwd Pkts` and the model could misinterpret it and gives more importance to the Income column)

In [32]:
scaler = StandardScaler()


numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

### Prepare data for Deep Learning (convert datas into float32)

In [33]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')


Y_train = np.array(Y_train).astype('float32')
Y_test = np.array(Y_test).astype('float32')

X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

joblib.dump(X_train, '../../preprocessed_data/CICIOT_update/X_train.joblib')
joblib.dump(X_test, '../../preprocessed_data/CICIOT_update/X_test.joblib')
joblib.dump(Y_train, '../../preprocessed_data/CICIOT_update/Y_train.joblib')
joblib.dump(Y_test, '../../preprocessed_data/CICIOT_update/Y_test.joblib')


['../../preprocessed_data/CICIOT_update/Y_test.joblib']